In [2]:
import pandas as pd
from openpyxl import load_workbook

# Путь к файлу
file_path = "aav.xlsx"

# Загружаем книгу (data_only=False, чтобы сохранить формулы)
wb = load_workbook(file_path, data_only=False)

# Функция для чтения листов с формулами
def read_sheet_with_formulas(sheet_name):
    sheet = wb[sheet_name]
    data = []

    for row in sheet.iter_rows():
        row_data = []
        for cell in row:
            # Проверяем, есть ли формула
            if cell.value and isinstance(cell.value, str) and cell.value.startswith("="):
                row_data.append(cell.value)  # Оставляем формулу как текст
            else:
                row_data.append(cell.value)  # Обычные значения
        data.append(row_data)

    return pd.DataFrame(data)

# Читаем все три листа
df_sheet1 = read_sheet_with_formulas("Sheet1")
df_plasmids = read_sheet_with_formulas("Плазмиды")
df_transduction = read_sheet_with_formulas("Трансдукция ААВ")

# Выводим первые строки каждого листа
print("\n=== Sheet1 ===")
print(df_sheet1.head())

print("\n=== Плазмиды ===")
print(df_plasmids.head())

print("\n=== Трансдукция ААВ ===")
print(df_transduction.head())

# Сохраняем в CSV (для удобства)
df_sheet1.to_csv("Sheet1.csv", index=False)
df_plasmids.to_csv("Plasmids.csv", index=False)
df_transduction.to_csv("Transduction_AAV.csv", index=False)



=== Sheet1 ===
                                            0      1             2   \
0                                         None   None          None   
1  перевод единиц объема наработки в объем КЖ    None          None   
2                              Единица объема   Т175   КРУГЛЫЕ 10см   
3                              Объем среды, мл     30            ND   
4                     Площадь поверхности, см2    175           100   

                  3     4     5                             6   \
0               None  None  None                          None   
1               None  None  None                          None   
2  5-level cellstack  None  None               Единица объема    
3                 ND  None  None               Объем среды, мл   
4               3180  None  None  Объем ТФ микса на флакон, мл   

                                        7             8                  9   \
0                                     None          None               None   
1 